# hello_milvus Demo

`hello_milvus.ipynb` demonstrates the basic operations of PyMilvus, a Python SDK of Milvus.
Before running, make sure that you have a running Milvus instance.

1. connect to Milvus
2. create collection
3. insert data
4. create index
5. search, query, and hybrid search on entities
6. delete entities by PK
7. drop collection

In [25]:
import numpy as np
import time

from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

fmt = "\n=== {:30} ===\n"
search_latency_fmt = "search latency = {:.4f}s"
num_entities, dim = 3000, 8

## 1. connect to Milvus

Add a new connection alias `default` for Milvus server in `localhost:19530`. 

Actually the `default` alias is a buildin in PyMilvus. If the address of Milvus is the same as `localhost:19530`, you can omit all parameters and call the method as: `connections.connect()`.

Note: the `using` parameter of the following methods is default to "default".

In [26]:
connections.connect("default", host="localhost", port="19530")

has = utility.has_collection("hello_milvus")
print(f"Does collection hello_milvus exist in Milvus: {has}")

Does collection hello_milvus exist in Milvus: False


## 2. create collection
We're going to create a collection with 3 fields.

|   |field name  |field type |other attributes              |  field description      |
|---|:----------:|:---------:|:----------------------------:|:-----------------------:|
|1  |    "pk"    |   VARCHAR |is_primary=True, auto_id=False|      "primary field"    |
|2  |  "random"  |   Double  |                              |      "a double field"   |
|3  |"embeddings"|FloatVector|     dim=8                    |"float vector with dim 8"|

In [27]:
fields = [
    FieldSchema(name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=False, max_length=100),
    FieldSchema(name="random", dtype=DataType.DOUBLE),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=dim)
]

schema = CollectionSchema(fields, "hello_milvus is the simplest demo to introduce the APIs")

hello_milvus = Collection("hello_milvus", schema, consistency_level="Strong")

## 3. insert data

We are going to insert 3000 rows of data into `hello_milvus`. Data to be inserted must be organized in fields.

The insert() method returns:
- either automatically generated primary keys by Milvus if auto_id=True in the schema;
- or the existing primary key field from the entities if auto_id=False in the schema.

In [28]:
rng = np.random.default_rng(seed=19530)
entities = [
    # provide the pk field because `auto_id` is set to False
    [str(i) for i in range(num_entities)],
    rng.random(num_entities).tolist(),  # field random, only supports list
    rng.random((num_entities, dim)),    # field embeddings, supports numpy.ndarray and list
]

insert_result = hello_milvus.insert(entities)

print(f"Number of entities in Milvus: {hello_milvus.num_entities}")  # check the num_entites

Number of entities in Milvus: 3000


## 4. create index
We are going to create an IVF_FLAT index for hello_milvus collection.

create_index() can only be applied to `FloatVector` and `BinaryVector` fields.

In [29]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}

hello_milvus.create_index("embeddings", index)

Status(code=0, message='')

## 5. search, query, and hybrid search
After data were inserted into Milvus and indexed, you can perform:
- search based on vector similarity
- query based on scalar filtering(boolean, int, etc.)
- hybrid search based on vector similarity and scalar filtering.

Before conducting a search or a query, you need to load the data in `hello_milvus` into memory.

In [30]:
hello_milvus.load()

**Search based on vector similarity**

In [31]:
vectors_to_search = entities[-1][-2:]
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}

start_time = time.time()
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, output_fields=["random"])
end_time = time.time()

for hits in result:
    for hit in hits:
        print(f"hit: {hit}, random field: {hit.entity.get('random')}")
print(search_latency_fmt.format(end_time - start_time))

hit: (distance: 0.0, id: 2998), random field: 0.9728033590489911
hit: (distance: 0.08883658051490784, id: 1262), random field: 0.2978858685751561
hit: (distance: 0.09590047597885132, id: 1265), random field: 0.3042039939240304
hit: (distance: 0.0, id: 2999), random field: 0.02316334456872482
hit: (distance: 0.05628091096878052, id: 1580), random field: 0.3855988746044062
hit: (distance: 0.08096685260534286, id: 2377), random field: 0.8745922204004368
search latency = 0.2968s


**Query based on scalar filtering(boolean, int, etc.)**

Start quering with `random > 0.5`

In [32]:
start_time = time.time()
result = hello_milvus.query(expr="random > 0.5", output_fields=["random", "embeddings"])
end_time = time.time()

print(f"query result:\n-{result[0]}")
print(search_latency_fmt.format(end_time - start_time))

query result:
-{'pk': '4', 'embeddings': [0.803092, 0.748499, 0.733152, 0.7597, 0.113219, 0.028412, 0.496197, 0.191172], 'random': 0.744296470467782}
search latency = 0.2481s


**Hybrid search**

Start hybrid searching with `random > 0.5`

In [33]:
start_time = time.time()
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, expr="random > 0.5", output_fields=["random"])
end_time = time.time()

for hits in result:
    for hit in hits:
        print(f"hit: {hit}, random field: {hit.entity.get('random')}")
print(search_latency_fmt.format(end_time - start_time))

hit: (distance: 0.0, id: 2998), random field: 0.9728033590489911
hit: (distance: 0.14606499671936035, id: 747), random field: 0.5648774800635661
hit: (distance: 0.1530652642250061, id: 2527), random field: 0.8928974315571507
hit: (distance: 0.08096685260534286, id: 2377), random field: 0.8745922204004368
hit: (distance: 0.20354536175727844, id: 2034), random field: 0.5526117606328499
hit: (distance: 0.21908017992973328, id: 958), random field: 0.6647383716417955
search latency = 0.1282s


## 6. delete entities by PK
You can delete entities by their PK values using boolean expressions.


In [34]:
ids = insert_result.primary_keys
expr = f'pk in ["{ids[0]}", "{ids[1]}"]'

result = hello_milvus.query(expr=expr, output_fields=["random", "embeddings"])
print(f"query before delete by expr=`{expr}` -> result: \n-{result[0]}\n-{result[1]}\n")

hello_milvus.delete(expr)

result = hello_milvus.query(expr=expr, output_fields=["random", "embeddings"])
print(f"query after delete by expr=`{expr}` -> result: {result}\n")

query before delete by expr=`pk in ["0", "1"]` -> result: 
-{'pk': '0', 'random': 0.6378742006852851, 'embeddings': [0.209635, 0.397467, 0.120191, 0.694749, 0.953557, 0.545455, 0.823604, 0.210963]}
-{'pk': '1', 'random': 0.43925103574669633, 'embeddings': [0.523236, 0.80354, 0.778247, 0.803696, 0.49148, 0.826561, 0.614527, 0.802345]}

query after delete by expr=`pk in ["0", "1"]` -> result: []



## 7. drop collection
Finally, drop the hello_milvus collection

In [35]:
utility.drop_collection("hello_milvus")